# 06 - Introducción a RAG (Retrieval-Augmented Generation)

## Curso de LLMs y Aplicaciones de IA

**Duración estimada:** 2.5-3 horas

---

## Índice

1. [¿Qué es RAG?](#intro)
2. [Arquitectura de un sistema RAG](#arquitectura)
3. [Implementación paso a paso](#implementacion)
4. [RAG con LangChain](#langchain)
5. [Tipos de memoria en RAG](#memoria)
6. [Mejores prácticas](#mejores)
7. [Ejercicios prácticos](#ejercicios)

---

## Objetivos de aprendizaje

Al finalizar este notebook, serás capaz de:
- Entender qué es RAG y cuándo usarlo
- Implementar un sistema RAG básico
- Integrar retrieval con generación de respuestas
- Añadir memoria conversacional al RAG
- Aplicar mejores prácticas de diseño

<a name="intro"></a>
## 1. ¿Qué es RAG?

**RAG** (Retrieval-Augmented Generation) es una técnica que combina:
1. **Retrieval**: Buscar información relevante en una base de conocimiento
2. **Augmented**: Añadir esa información al contexto del LLM
3. **Generation**: Generar respuestas basadas en el contexto aumentado

### ¿Por qué RAG?

| Problema del LLM | Solución con RAG |
|-----------------|------------------|
| Conocimiento desactualizado | Acceso a información actual |
| No conoce datos privados | Acceso a documentos corporativos |
| Alucinaciones | Respuestas basadas en fuentes |
| Sin citación | Puede citar fuentes |

### RAG vs Fine-tuning

| Aspecto | RAG | Fine-tuning |
|---------|-----|-------------|
| Costo | Bajo | Alto |
| Actualización | Fácil | Requiere re-entrenar |
| Explicabilidad | Alta (cita fuentes) | Baja |
| Conocimiento privado | ✅ | ✅ |
| Latencia | Mayor | Menor |

In [1]:
# Install required libraries (all free)
#!pip install -q langchain langchain-community langchain-groq langchain-huggingface
#!pip install -q faiss-cpu sentence-transformers

In [2]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

# Setup Groq API (FREE tier)
if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass("Introduce tu GROQ API Key: ")

print("Configuración completada ✓")

Introduce tu GROQ API Key:  ········


Configuración completada ✓


<a name="arquitectura"></a>
## 2. Arquitectura de un sistema RAG

```
┌─────────────────────────────────────────────────────────────────┐
│                      FASE DE INDEXACIÓN                         │
│  Documentos → [Loader] → [Splitter] → [Embeddings] → VectorDB  │
└─────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────┐
│                      FASE DE CONSULTA                           │
│  Query → [Embedding] → [Retrieval] → Contexto                  │
│                                          ↓                      │
│  Query + Contexto → [LLM] → Respuesta                          │
└─────────────────────────────────────────────────────────────────┘
```

<a name="implementacion"></a>
## 3. Implementación paso a paso

Vamos a construir un RAG para responder preguntas sobre una base de conocimiento.

In [3]:
# Step 1: Create knowledge base
from langchain_core.documents import Document

# Sample knowledge base about a fictional company
knowledge_base = [
    Document(
        page_content="TechCorp fue fundada en 2015 en Madrid por María García y Juan López. "
                     "La empresa se especializa en soluciones de inteligencia artificial para empresas.",
        metadata={"source": "about.txt", "section": "historia"}
    ),
    Document(
        page_content="Los productos principales de TechCorp incluyen: AIAssist (asistente virtual), "
                     "DataVision (análisis de datos), y CloudBrain (procesamiento en la nube).",
        metadata={"source": "products.txt", "section": "productos"}
    ),
    Document(
        page_content="TechCorp tiene oficinas en Madrid (sede central), Barcelona y Valencia. "
                     "Actualmente cuenta con 150 empleados.",
        metadata={"source": "about.txt", "section": "oficinas"}
    ),
    Document(
        page_content="El horario de atención al cliente es de lunes a viernes, de 9:00 a 18:00. "
                     "Email de soporte: soporte@techcorp.es. Teléfono: 900 123 456.",
        metadata={"source": "contact.txt", "section": "contacto"}
    ),
    Document(
        page_content="TechCorp ofrece planes de precios: Básico (99€/mes), Profesional (299€/mes), "
                     "y Enterprise (personalizado). Todos incluyen soporte técnico.",
        metadata={"source": "pricing.txt", "section": "precios"}
    ),
    Document(
        page_content="La política de devoluciones permite cancelar en los primeros 30 días "
                     "con reembolso completo. Después, se aplica un cargo del 20%.",
        metadata={"source": "policies.txt", "section": "devoluciones"}
    ),
]

print(f"Base de conocimiento creada: {len(knowledge_base)} documentos")

Base de conocimiento creada: 6 documentos


In [4]:
# Step 2: Create embeddings and vector store
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Load free embedding model
print("Cargando modelo de embeddings...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Create vector store
print("Creando vector store...")
vector_store = FAISS.from_documents(knowledge_base, embeddings)

print(f"Vector store creado con {vector_store.index.ntotal} vectores")

Cargando modelo de embeddings...


Loading weights: 100%|████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 1070.65it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Creando vector store...
Vector store creado con 6 vectores


In [5]:
# Step 3: Create retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Return top 3 documents
)

# Test retriever
test_query = "¿Cuáles son los precios?"
retrieved_docs = retriever.invoke(test_query)

print(f"Query: '{test_query}'")
print(f"\nDocumentos recuperados: {len(retrieved_docs)}")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n{i}. {doc.page_content[:80]}...")

Query: '¿Cuáles son los precios?'

Documentos recuperados: 3

1. TechCorp tiene oficinas en Madrid (sede central), Barcelona y Valencia. Actualme...

2. La política de devoluciones permite cancelar en los primeros 30 días con reembol...

3. TechCorp ofrece planes de precios: Básico (99€/mes), Profesional (299€/mes), y E...


In [6]:
# Step 4: Setup LLM
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.3
)

print("LLM configurado ✓")

LLM configurado ✓


In [7]:
# Step 5: Create RAG prompt
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente de atención al cliente de TechCorp.
    
Responde las preguntas ÚNICAMENTE basándote en el contexto proporcionado.
Si la información no está en el contexto, di que no tienes esa información.
Sé conciso y profesional.

Contexto:
{context}"""),
    ("human", "{question}")
])

print("Prompt RAG configurado ✓")

Prompt RAG configurado ✓


In [8]:
# Step 6: Create RAG chain (manual implementation)
def rag_answer(question: str) -> dict:
    """Answer a question using RAG."""
    # 1. Retrieve relevant documents
    docs = retriever.invoke(question)
    
    # 2. Format context
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # 3. Create prompt with context
    messages = rag_prompt.format_messages(
        context=context,
        question=question
    )
    
    # 4. Get LLM response
    response = llm.invoke(messages)
    
    return {
        "question": question,
        "answer": response.content,
        "sources": [doc.metadata.get("source", "unknown") for doc in docs]
    }

# Test the RAG system
result = rag_answer("¿Cuándo se fundó la empresa y quiénes son los fundadores?")

print(f"Pregunta: {result['question']}")
print(f"\nRespuesta: {result['answer']}")
print(f"\nFuentes: {result['sources']}")

Pregunta: ¿Cuándo se fundó la empresa y quiénes son los fundadores?

Respuesta: La empresa TechCorp se fundó en 2015 en Madrid. Los fundadores son María García y Juan López.

Fuentes: ['about.txt', 'about.txt', 'pricing.txt']


In [9]:
# Test with multiple questions
questions = [
    "¿Cuáles son los productos de TechCorp?",
    "¿Cómo puedo contactar con soporte?",
    "¿Cuál es la política de devoluciones?",
    "¿Cuántos empleados tiene la empresa?",
    "¿Ofrecen servicios de marketing?"  # Not in knowledge base
]

print("Pruebas del sistema RAG")
print("=" * 60)

for q in questions:
    result = rag_answer(q)
    print(f"\n❓ {result['question']}")
    print(f"💬 {result['answer']}")
    print("-" * 40)

Pruebas del sistema RAG

❓ ¿Cuáles son los productos de TechCorp?
💬 Los productos principales de TechCorp son:

1. AIAssist (asistente virtual)
2. DataVision (análisis de datos)
3. CloudBrain (procesamiento en la nube)
----------------------------------------

❓ ¿Cómo puedo contactar con soporte?
💬 Puedes contactar con nuestro soporte a través del teléfono 900 123 456 o enviando un correo electrónico a soporte@techcorp.es. Nuestro horario de atención es de lunes a viernes, de 9:00 a 18:00.
----------------------------------------

❓ ¿Cuál es la política de devoluciones?
💬 La política de devoluciones de TechCorp permite cancelar en los primeros 30 días con reembolso completo. Después de este plazo, se aplica un cargo del 20%.
----------------------------------------

❓ ¿Cuántos empleados tiene la empresa?
💬 La empresa TechCorp cuenta con 150 empleados.
----------------------------------------

❓ ¿Ofrecen servicios de marketing?
💬 No tengo esa información. Solo sé que se especializan en 

<a name="langchain"></a>
## 4. RAG con LangChain (Simplificado)

LangChain proporciona componentes que simplifican la creación de chains RAG.

In [11]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# Create the document chain
system_prompt = """Eres un asistente de TechCorp. Responde basándote en el contexto.
Si no tienes la información, dilo claramente.

Contexto:
{context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Create chains
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("RAG Chain creada con LangChain ✓")

RAG Chain creada con LangChain ✓


In [12]:
# Test the chain
response = rag_chain.invoke({"input": "¿Dónde están las oficinas de TechCorp?"})

print(f"Pregunta: {response['input']}")
print(f"\nRespuesta: {response['answer']}")
print(f"\nContexto usado ({len(response['context'])} documentos):")
for doc in response['context']:
    print(f"  - {doc.page_content[:60]}...")

Pregunta: ¿Dónde están las oficinas de TechCorp?

Respuesta: Las oficinas de TechCorp están ubicadas en tres ciudades de España: 

1. Madrid (sede central)
2. Barcelona
3. Valencia

Contexto usado (3 documentos):
  - TechCorp fue fundada en 2015 en Madrid por María García y Ju...
  - TechCorp tiene oficinas en Madrid (sede central), Barcelona ...
  - TechCorp ofrece planes de precios: Básico (99€/mes), Profesi...


<a name="memoria"></a>
## 5. RAG con Memoria Conversacional

Añadir memoria permite que el RAG recuerde el contexto de la conversación.

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Create a store for session histories
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Update prompt to include chat history
contextualize_prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente de TechCorp. Responde basándote en el contexto.
    
Contexto de la base de conocimiento:
{context}"""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}")
])

# Create chain with history
qa_chain = create_stuff_documents_chain(llm, contextualize_prompt)
rag_chain_with_history = create_retrieval_chain(retriever, qa_chain)

# Wrap with message history
conversational_rag = RunnableWithMessageHistory(
    rag_chain_with_history,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

print("RAG conversacional configurado ✓")

RAG conversacional configurado ✓


In [14]:
# Test conversational RAG
session_config = {"configurable": {"session_id": "user123"}}

# First question
response1 = conversational_rag.invoke(
    {"input": "¿Cuáles son los planes de precios?"},
    config=session_config
)
print(f"Q1: ¿Cuáles son los planes de precios?")
print(f"A1: {response1['answer']}\n")

# Follow-up question (should use context from previous)
response2 = conversational_rag.invoke(
    {"input": "¿Y el más barato incluye soporte?"},
    config=session_config
)
print(f"Q2: ¿Y el más barato incluye soporte?")
print(f"A2: {response2['answer']}")

Q1: ¿Cuáles son los planes de precios?
A1: Nuestros planes de precios son los siguientes:

1. **Básico**: 99€/mes
2. **Profesional**: 299€/mes
3. **Enterprise**: Personalizado (el precio se adapta a las necesidades específicas de cada cliente)

Todos nuestros planes incluyen soporte técnico. ¿Necesitas más información sobre alguno de ellos?

Q2: ¿Y el más barato incluye soporte?
A2: Sí, el plan **Básico** incluye soporte técnico limitado. Esto significa que podrás acceder a nuestro centro de ayuda en línea y recibirás respuesta a tus consultas técnicas dentro de un plazo de 24 horas hábiles.

Si necesitas un soporte técnico más avanzado o prioritario, te recomiendo considerar el plan **Profesional**, que incluye soporte técnico prioritario y acceso a un equipo de expertos que pueden ayudarte a resolver tus dudas de manera más rápida y personalizada.


<a name="mejores"></a>
## 6. Mejores Prácticas

### Optimización de Retrieval

| Técnica | Descripción | Cuándo usar |
|---------|-------------|-------------|
| **Aumentar k** | Recuperar más documentos | Respuestas incompletas |
| **Chunk overlap** | Solapar fragmentos | Contexto cortado |
| **Reranking** | Reordenar resultados | Precisión crítica |
| **Hybrid search** | Combinar keyword + semántico | Términos técnicos |

In [15]:
# Example: RAG with source citation
def rag_with_sources(question: str) -> str:
    """Answer with source citations."""
    # Retrieve
    docs = retriever.invoke(question)
    
    # Format context with numbered sources
    context_parts = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'unknown')
        context_parts.append(f"[{i}] ({source}): {doc.page_content}")
    
    context = "\n\n".join(context_parts)
    
    # Create prompt
    prompt = f"""Responde la pregunta basándote en el contexto.
Incluye las referencias [1], [2], etc. cuando cites información.

Contexto:
{context}

Pregunta: {question}

Respuesta:"""
    
    response = llm.invoke(prompt)
    return response.content

# Test
answer = rag_with_sources("¿Cuándo se fundó TechCorp y dónde tiene oficinas?")
print(answer)

Según la información disponible [1] y [2], TechCorp se fundó en 2015 en Madrid por María García y Juan López. Actualmente, la empresa tiene oficinas en Madrid (sede central), Barcelona y Valencia.


<a name="ejercicios"></a>
## 7. Ejercicios Prácticos

### Ejercicio 1: RAG para documentación técnica

In [16]:
# Exercise 1: Create a RAG for API documentation

api_docs = [
    Document(page_content="GET /users - Obtiene lista de usuarios. Parámetros: limit (int), offset (int).",
             metadata={"endpoint": "/users", "method": "GET"}),
    Document(page_content="POST /users - Crea un nuevo usuario. Body: {name: string, email: string}.",
             metadata={"endpoint": "/users", "method": "POST"}),
    Document(page_content="GET /users/{id} - Obtiene un usuario por ID. Retorna 404 si no existe.",
             metadata={"endpoint": "/users/{id}", "method": "GET"}),
    Document(page_content="Autenticación: Todas las peticiones requieren header 'Authorization: Bearer <token>'.",
             metadata={"section": "auth"}),
    Document(page_content="Errores comunes: 401 (no autorizado), 404 (no encontrado), 500 (error servidor).",
             metadata={"section": "errors"}),
]

# Create your RAG system for API docs
# api_vs = FAISS.from_documents(api_docs, embeddings)
# api_retriever = api_vs.as_retriever(search_kwargs={"k": 2})

# Test questions:
# - "¿Cómo creo un usuario nuevo?"
# - "¿Qué significa el error 401?"
# - "¿Cómo me autentico?"

### Ejercicio 2: RAG con filtrado por metadatos

In [17]:
# Exercise 2: RAG with metadata filtering
# Create a system that can filter by document section

def rag_with_filter(question: str, section_filter: str = None) -> str:
    """RAG with optional section filtering."""
    # Get more results to allow filtering
    docs = retriever.invoke(question)
    
    # Filter by section if specified
    if section_filter:
        docs = [d for d in docs if d.metadata.get("section") == section_filter]
    
    if not docs:
        return "No encontré información relevante en esa sección."
    
    context = "\n".join([d.page_content for d in docs])
    
    # Use LLM to answer
    prompt = f"Contexto: {context}\n\nPregunta: {question}\nRespuesta:"
    response = llm.invoke(prompt)
    return response.content

# Test
# print(rag_with_filter("¿Cuál es el horario?", section_filter="contacto"))

## Resumen

En este notebook hemos aprendido:

1. **RAG**: Combina retrieval + LLM para respuestas informadas
2. **Arquitectura**: Indexación → Retrieval → Generation
3. **Implementación**: Paso a paso y con LangChain
4. **Memoria**: RAG conversacional con historial
5. **Citación**: Incluir fuentes en respuestas

### Flujo típico de RAG

```
1. Usuario hace pregunta
2. Pregunta → Embedding → Buscar en Vector Store
3. Recuperar documentos relevantes
4. Construir prompt con contexto
5. LLM genera respuesta basada en contexto
6. (Opcional) Añadir fuentes/citaciones
```

En el siguiente notebook veremos **Reranking** para mejorar la calidad del retrieval.

---

## Referencias

- [RAG Paper (Lewis et al., 2020)](https://arxiv.org/abs/2005.11401)
- [LangChain RAG Guide](https://python.langchain.com/docs/tutorials/rag/)
- [RAG Best Practices](https://www.pinecone.io/learn/retrieval-augmented-generation/)

In [18]:
import session_info
session_info.show(html = False)

-----
ipykernel                   7.2.0
langchain_classic           1.0.1
langchain_community         0.4.1
langchain_core              1.2.9
langchain_groq              1.1.2
langchain_huggingface       NA
session_info                v1.0.1
-----
IPython             9.10.0
jupyter_client      8.8.0
jupyter_core        5.9.1
-----
Python 3.13.1 (tags/v3.13.1:0671451, Dec  3 2024, 19:06:28) [MSC v.1942 64 bit (AMD64)]
Windows-11-10.0.26200-SP0
-----
Session information updated at 2026-02-09 17:09
